In [48]:
import numpy as np
import sklearn as skl
from numpy import genfromtxt, matlib
from sklearn import kernel_ridge
import os
from sklearn.model_selection import GridSearchCV
print(len(os.sched_getaffinity(0)))

16


In [49]:
path_to_train = '/home/sathvikc/AMATH-482-2/AMATH-482/Homework Assignments/Homework 3/wine_training.csv'
path_to_test = '/home/sathvikc/AMATH-482-2/AMATH-482/Homework Assignments/Homework 3/wine_test.csv'

train = genfromtxt(path_to_train, delimiter=',')
test = genfromtxt(path_to_test, delimiter=',')

print('TRAIN SET')
print(train)
print('ALL BUT LAST')
print(train[:, :-1])
print('LAST')
print(train[:, -1])

TRAIN SET
[[ 6.1   0.6   0.08 ...  0.54 11.    5.  ]
 [10.3   0.53  0.48 ...  0.59  9.3   6.  ]
 [ 6.9   0.49  0.19 ...  0.64  9.8   6.  ]
 ...
 [ 7.    0.38  0.49 ...  0.77 11.4   6.  ]
 [10.    0.35  0.47 ...  0.52 12.    6.  ]
 [ 7.    0.6   0.3  ...  1.17 10.2   5.  ]]
ALL BUT LAST
[[ 6.1   0.6   0.08 ...  3.38  0.54 11.  ]
 [10.3   0.53  0.48 ...  3.12  0.59  9.3 ]
 [ 6.9   0.49  0.19 ...  3.38  0.64  9.8 ]
 ...
 [ 7.    0.38  0.49 ...  3.39  0.77 11.4 ]
 [10.    0.35  0.47 ...  3.23  0.52 12.  ]
 [ 7.    0.6   0.3  ...  3.3   1.17 10.2 ]]
LAST
[5. 6. 6. ... 6. 6. 5.]


In [50]:
X_train_N = train[:, :-1].shape[0]

X_train_mean = np.mean(train[:, :-1], axis=0)
X_train_std = np.std(train[:, :-1], axis=0)
X_train_normal = (train[:, :-1] - matlib.repmat(X_train_mean, X_train_N, 1))/matlib.repmat(X_train_std, X_train_N, 1)

Y_train_N = train[:, -1].shape[0]

Y_train_mean = np.mean(train[:, -1], axis=0)
Y_train_std = np.std(train[:, -1], axis=0)

Y_train_normal = (train[:, -1] - Y_train_mean)/Y_train_std

In [51]:
X_test_N = test[:, :-1].shape[0]

# X_test_mean = np.mean(test[:, :-1], axis=0)
# X_test_std = np.std(test[:, :-1], axis=0)

X_test_normal = (test[:, :-1] - matlib.repmat(X_train_mean, X_test_N, 1))/matlib.repmat(X_train_std, X_test_N, 1)

Y_test_N = test[:, -1].shape[0]

# Y_test_mean = np.mean(test[:, -1], axis=0)
# Y_test_std = np.std(test[:, -1], axis=0)

Y_test_normal = (test[:, -1] - Y_train_mean)/Y_train_std

In [52]:
#Use linear regression to fit a linear model to the training set 
LinearRegressor = skl.linear_model.LinearRegression()
LinearRegressor.fit(X_train_normal, Y_train_normal)
predicted_train = LinearRegressor.predict(X_train_normal)
print(f'LINEAR REGRESSION TRAINING MSE: {skl.metrics.mean_squared_error(predicted_train, Y_train_normal)}')
predicted_test = LinearRegressor.predict(X_test_normal)
print(f'LINEAR REGRESSION TEST MSE: {skl.metrics.mean_squared_error(predicted_test, Y_test_normal)}')

LINEAR REGRESSION TRAINING MSE: 0.6278484956554882
LINEAR REGRESSION TEST MSE: 0.747169690518721


In [53]:
#Use kernel ridge regression to fit a non-linear model to the training set

#Gaussian (RBF) Kernel 
RBFRegresor = kernel_ridge.KernelRidge(kernel = 'rbf')  

#Laplacian kernel
LaplacianRegresor = kernel_ridge.KernelRidge(kernel = 'laplacian')

In [63]:
# range of values of sigma to try 
K_sgm = 10
K_lmbd = 10

sgm = np.linspace(1, 3, K_sgm)
lmbd = np.linspace(-3, -1, K_lmbd)

#RBF 
cv = GridSearchCV(RBFRegresor, param_grid = {
    'alpha': 2 ** lmbd,
    'gamma': 1 / (2*(2**sgm)**2)
    }, cv = 10, scoring= 'neg_mean_squared_error', n_jobs = -1)

cv.fit(X_train_normal, Y_train_normal)
print(cv.best_params_)

{'alpha': 0.19842513149602492, 'gamma': 0.03645403248675365}


In [64]:
best_alpha_rbf = cv.best_params_['alpha']
best_gamma_rbf = cv.best_params_['gamma']
print(best_alpha_rbf)
print(best_gamma_rbf)

0.19842513149602492
0.03645403248675365


In [56]:
# # range of values of sigma to try 


# K_sgm = 10
# K_lmbd = 10

# sgm = np.linspace(1, 3, K_sgm)
# lmbd = np.linspace(-3, -1, K_lmbd)

# scores = np.zeros((K_sgm, K_lmbd))
# scores_std = np.zeros((K_sgm, K_lmbd))

# KRR_CV = skl.kernel_ridge.KernelRidge(kernel='rbf')

# for i in range(K_sgm):

#   KRR_CV.gamma = 1/(2*(2**sgm[i])**2)

#   for j in range(K_lmbd): 

#     KRR_CV.alpha = (2**lmbd[j])
#     this_score = skl.model_selection.cross_val_score(KRR_CV, X_train_normal, Y_train_normal, scoring= 'neg_mean_squared_error', cv = 10)

#     scores[i,j] = (np.mean(this_score))
#     scores_std[i,j] = (np.std(this_score))

# print(scores)

# ij_max = np.array( np.where( scores == scores.max() ), dtype=int).flatten()

# print(ij_max)

# print(scores.max())

# print('log_2 sg:', sgm[ij_max[0]], 'log_2_lmbd: ', lmbd[ij_max[1]])
# print('alpha: ', 2**lmbd[ij_max[1]])
# print('gamma: ', 1/(2*(2**sgm[ij_max[0]])**2))
# best_alpha_rbf = 2**lmbd[ij_max[1]]
# best_gamma_rbf = 1/(2*(2**sgm[ij_max[0]])**2)

In [57]:
# range of values of sigma to try 
K_sgm = 10
K_lmbd = 10

sgm = np.linspace(-4, 4, K_sgm)
lmbd = np.linspace(-4, 4, K_lmbd)

#Laplacian
cv = GridSearchCV(LaplacianRegresor, param_grid = {
    'alpha': 2 ** lmbd,
    'gamma': 1 / (2 ** (sgm))
    }, cv = 5, scoring= 'neg_mean_squared_error', n_jobs=-1)

cv.fit(X_train_normal, Y_train_normal)

GridSearchCV(cv=5, estimator=KernelRidge(kernel='laplacian'), n_jobs=-1,
             param_grid={'alpha': array([ 0.0625    ,  0.11573434,  0.214311  ,  0.39685026,  0.73486725,
        1.36079   ,  2.5198421 ,  4.66611616,  8.64047791, 16.        ]),
                         'gamma': array([16.        ,  8.64047791,  4.66611616,  2.5198421 ,  1.36079   ,
        0.73486725,  0.39685026,  0.214311  ,  0.11573434,  0.0625    ])},
             scoring='neg_mean_squared_error')

In [58]:
best_alpha_laplacian = cv.best_params_['alpha']
best_gamma_laplacian = cv.best_params_['gamma']
print(best_alpha_laplacian)
print(best_gamma_laplacian)

0.2143109957132682
0.21431099571326834


In [59]:
# range of values of sigma to try 


# K_sgm = 10
# K_lmbd = 10

# sgm = np.linspace(0, 4, K_sgm)
# lmbd = np.linspace(-3, 1, K_lmbd)

# scores = np.zeros((K_sgm, K_lmbd))
# scores_std = np.zeros((K_sgm, K_lmbd))

# KRR_CV = skl.kernel_ridge.KernelRidge(kernel = 'laplacian')

# for i in range(K_sgm):

#   KRR_CV.gamma = 1 / (2 ** (sgm[i]))

#   for j in range(K_lmbd): 

#     KRR_CV.alpha = (2**lmbd[j])
#     this_score = skl.model_selection.cross_val_score(KRR_CV, X_train_normal, Y_train_normal, scoring= 'neg_mean_squared_error', cv = 10)

#     scores[i,j] = (np.mean(this_score))
#     scores_std[i,j] = (np.std(this_score))

# print(scores)

# ij_max = np.array( np.where( scores == scores.max() ), dtype=int).flatten()

# print(ij_max)

# print(scores.max())

# print('log_2 sg:', sgm[ij_max[0]], 'log_2_lmbd: ', lmbd[ij_max[1]])
# print('alpha: ', 2**lmbd[ij_max[1]])
# print('gamma: ', 1/(2**sgm[ij_max[0]]))
# best_alpha_laplacian = 2**lmbd[ij_max[1]]
# best_gamma_laplacian = 1/(2*(2**sgm[ij_max[0]])**2)

In [60]:
#MSE of all 3 models

#Linear Regression
print(f'LINEAR REGRESSION TRAINING MSE: {skl.metrics.mean_squared_error(predicted_train, Y_train_normal)}')
print(f'LINEAR REGRESSION TEST MSE: {skl.metrics.mean_squared_error(predicted_test, Y_test_normal)}')

#RBF
RBFRegresor = kernel_ridge.KernelRidge(alpha = best_alpha_rbf, kernel = 'rbf', gamma = best_gamma_rbf)
RBFRegresor.fit(X_train_normal, Y_train_normal)
predicted_train = RBFRegresor.predict(X_train_normal)
print(f'GAUSSIAN KERNEL TRAINING MSE: {skl.metrics.mean_squared_error(predicted_train, Y_train_normal)}')
predicted_test = RBFRegresor.predict(X_test_normal)
print(f'GAUSSIAN KERNEL TEST MSE: {skl.metrics.mean_squared_error(predicted_test, Y_test_normal)}')

#Laplacian
LaplacianRegresor = kernel_ridge.KernelRidge(alpha = best_alpha_laplacian, kernel = 'laplacian', gamma = best_gamma_laplacian)
LaplacianRegresor.fit(X_train_normal, Y_train_normal)
predicted_train = LaplacianRegresor.predict(X_train_normal)
print(f'LAPLACIAN KERNEL TRAINING MSE: {skl.metrics.mean_squared_error(predicted_train, Y_train_normal)}')
predicted_test = LaplacianRegresor.predict(X_test_normal)
print(f'LAPLACIAN KERNEL TEST MSE: {skl.metrics.mean_squared_error(predicted_test, Y_test_normal)}')

LINEAR REGRESSION TRAINING MSE: 0.6278484956554882
LINEAR REGRESSION TEST MSE: 0.747169690518721
GAUSSIAN KERNEL TRAINING MSE: 0.4443644835200732
GAUSSIAN KERNEL TEST MSE: 0.6707146529560462
LAPLACIAN KERNEL TRAINING MSE: 0.058153392810602604
LAPLACIAN KERNEL TEST MSE: 0.6095835762585938


In [61]:
#Predict on new batch of wines 
path_to_new_wine = '/home/sathvikc/AMATH-482-2/AMATH-482/Homework Assignments/Homework 3/wine_new_batch.csv'
new_wine = genfromtxt(path_to_new_wine, delimiter=',')

X_new_wine_N = new_wine.shape[0]

# X_new_wine_mean = np.mean(new_wine, axis=0)
# X_new_wine_std = np.std(new_wine, axis=0)

X_new_wine_normal = (new_wine - matlib.repmat(X_train_mean, X_new_wine_N, 1))/matlib.repmat(X_train_std, X_new_wine_N, 1)

In [62]:
#Linear Regression 
linear_predict = LinearRegressor.predict(X_new_wine_normal)

#RBF
rbf_predict = RBFRegresor.predict(X_new_wine_normal)

#Laplacian
laplacian_predict = LaplacianRegresor.predict(X_new_wine_normal)

#De-normalize the predictions
linear_predict = (linear_predict * Y_train_std) + Y_train_mean
rbf_predict = (rbf_predict * Y_train_std) + Y_train_mean
laplacian_predict = (laplacian_predict * Y_train_std) + Y_train_mean

#print the predictions 
print(f'LINEAR PREDICTION: {linear_predict}')
print(f'RBF PREDICTION: {rbf_predict}')
print(f'LAPLACIAN PREDICTION: {laplacian_predict}')

LINEAR PREDICTION: [6.00469789 5.28767761 5.56363072 6.067022   5.94248207]
RBF PREDICTION: [6.02506681 5.45131357 5.42488008 6.18782517 6.1157892 ]
LAPLACIAN PREDICTION: [6.06872311 5.45686107 5.63144745 6.00113834 6.03145372]
